In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import SGD

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')

### Step 1 & 2: Data acquisition and preparation

In [177]:
def import_data(paths):
    """ Extracts features and labels from a dataframe.

    Args: 
        path: A list of paths to a collection of .csv's in the following dataset:
            https://archive.ics.uci.edu/ml/datasets/Character+Font+Images

    Returns:
        X: A #samples x 20 x 20 numpy array containing the pixel data
           for the font samples. The pixel values are originally grayscale
           [0, 255] but scaled to the range [0, 1].

        y: A numpy array containing the ascii code for each of the sample images.
    """
    cols = ["r"+str(x)+"c"+str(y) for x in range(20) for y in range(20)]
    dfs = []
    for path in paths:
        df = pd.read_csv(path)
        dfs.append(df)
    df = pd.concat(dfs, ignore_index=True)
    features = df[cols].copy()
    scale = np.vectorize(lambda x: x / 255)
    X = scale(np.reshape(features.values, features.shape[0] * features.shape[1]))
    X = np.reshape(X, (-1, 20, 20, 1))
    y = df['m_label'].copy().as_matrix()
    return X, y


def convert_to_one_hot(y, unique_chars):
    """ Returns a nSamples x nUniqueCharacters array which is a 
        one-hot representation of y.

    Args:
        y: A 1D array containing categorical labels.
        
        unique_chars: a list of the possible values in y.

    Returns:
        one_hot_rep: a one-hot representation of y.
    """
    one_hot_rep = []
    char_to_ix = { ch:i for i,ch in enumerate(unique_chars) }
    for label in y:
        one_hot = [0 for x in range(len(unique_chars))]
        one_hot[char_to_ix[label]] = 1
        one_hot_rep.append(one_hot)
    return one_hot_rep

### Step 3: Build a keras network

In [188]:
data_paths = ["fonts/AGENCY.csv"]
X, y = import_data(data_paths)
unique_chars = list(set(y))
y = convert_to_one_hot(y, unique_chars)

model = Sequential()
# Layer 1: Convolution 2D layer with relu activations
model.add(Conv2D(32, (4, 4), activation='relu', input_shape=(20, 20, 1)))
# Layer 2: Max pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# Layer 3: Convolution
model.add(Conv2D(64, (4, 4), activation='relu'))
# Layer 4: Max pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# Layer 5: Flatten
model.add(Flatten())
# Layer 6: Dropout
model.add(Dropout(0.4))
# Layer 7: Dense with relu activation
model.add(Dense(1024, activation='relu'))
# Layer 8: Dense with softmax activation
model.add(Dense(len(unique_chars), activation='softmax'))

### Step 4: Exploration and Evaluation

Evaluate the network using cross validation (splitting data into training/testing). What is its accuracy?

In [189]:
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.3)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(np.array(x_train), np.array(y_train), epochs=50, batch_size=32)  
score = model.evaluate(np.array(x_train), np.array(y_train), batch_size=32)
print("Train accuracy:", score[1])
score = model.evaluate(np.array(x_test), np.array(y_test), batch_size=32)
print("Test accuracy:", score[1])

Epoch 1/50
702/702 [==============================] - 3s 4ms/step - loss: 5.5386 - acc: 0.0014
Epoch 2/50
702/702 [==============================] - 0s 584us/step - loss: 5.4475 - acc: 0.0214
Epoch 3/50
702/702 [==============================] - 0s 598us/step - loss: 4.9442 - acc: 0.0641
Epoch 4/50
702/702 [==============================] - 0s 588us/step - loss: 4.0023 - acc: 0.1496
Epoch 5/50
702/702 [==============================] - 0s 621us/step - loss: 3.1377 - acc: 0.2593 0s - loss: 3.0619 - acc: 0.
Epoch 6/50
702/702 [==============================] - 0s 615us/step - loss: 2.5994 - acc: 0.3276
Epoch 7/50
702/702 [==============================] - 0s 598us/step - loss: 2.2018 - acc: 0.3932
Epoch 8/50
702/702 [==============================] - 0s 605us/step - loss: 1.9504 - acc: 0.4330
Epoch 9/50
702/702 [==============================] - 0s 632us/step - loss: 1.8145 - acc: 0.4530 0s - loss: 1.5618 - acc: 
Epoch 10/50
702/702 [==============================] - 0s 624us/step - loss

Create and train a different network topology (add more convolution/dropout layers, explore other types/sizes of layer). Try to find a topology that works better than the one described above.

In [172]:
data_paths = ["fonts/AGENCY.csv"]
X, y = import_data(data_paths)
unique_chars = list(set(y))
y = convert_to_one_hot(y, unique_chars)

model = Sequential()
# Layer 1: Convolution 2D layer with relu activations
model.add(Conv2D(32, (4, 4), activation='relu', input_shape=(20, 20, 1)))
# Layer 2: Max pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# Layer 3: Convolution
model.add(Conv2D(64, (4, 4), activation='relu'))
# Layer 4: Max pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# Layer 5: Flatten
model.add(Flatten())
# Layer 6: Dropout
model.add(Dropout(0.4))
# Layer 7: Dense with relu activation
model.add(Dense(1024, activation='relu'))
# Layer 8: Dense with softmax activation
model.add(Dense(len(unique_chars), activation='softmax'))

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.3)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(np.array(x_train), np.array(y_train), epochs=50, batch_size=32)  
score = model.evaluate(np.array(x_train), np.array(y_train), batch_size=32)
print("Train accuracy:", score[1])
score = model.evaluate(np.array(x_test), np.array(y_test), batch_size=32)
print("Test accuracy:", score[1])

Epoch 1/50
702/702 [==============================] - 2s 4ms/step - loss: 5.5435 - acc: 0.0014
Epoch 2/50
702/702 [==============================] - 0s 610us/step - loss: 5.4492 - acc: 0.0128
Epoch 3/50
702/702 [==============================] - 0s 598us/step - loss: 5.0451 - acc: 0.0370
Epoch 4/50
702/702 [==============================] - 1s 745us/step - loss: 4.2595 - acc: 0.1211
Epoch 5/50
702/702 [==============================] - 0s 642us/step - loss: 3.4957 - acc: 0.1781
Epoch 6/50
702/702 [==============================] - 1s 760us/step - loss: 2.9370 - acc: 0.2578
Epoch 7/50
702/702 [==============================] - 1s 737us/step - loss: 2.5634 - acc: 0.3248
Epoch 8/50
702/702 [==============================] - 0s 656us/step - loss: 2.2813 - acc: 0.3803
Epoch 9/50
702/702 [==============================] - 1s 745us/step - loss: 2.0608 - acc: 0.3974
Epoch 10/50
702/702 [==============================] - 0s 629us/step - loss: 1.8901 - acc: 0.4288
Epoch 11/50
702/702 [==========

Test the accuracy of your network with character inputs from a DIFFERENT font set. How does it perform?

In [181]:
train_paths = ["fonts/AGENCY.csv"]
test_paths = ["fonts/BELL.csv"]
x_train, y_train = import_data(train_paths)
x_test, y_test = import_data(test_paths)
unique_chars = list(set(y_train).union(y_test))
y_train = convert_to_one_hot(y_train, unique_chars)
y_test = convert_to_one_hot(y_test, unique_chars)

model = Sequential()
# Layer 1: Convolution 2D layer with relu activations
model.add(Conv2D(32, (4, 4), activation='relu', input_shape=(20, 20, 1)))
# Layer 2: Max pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# Layer 3: Convolution
model.add(Conv2D(64, (4, 4), activation='relu'))
# Layer 4: Max pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# Layer 5: Flatten
model.add(Flatten())
# Layer 6: Dropout
model.add(Dropout(0.4))
# Layer 7: Dense with relu activation
model.add(Dense(1024, activation='relu'))
# Layer 8: Dense with softmax activation
model.add(Dense(len(unique_chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(np.array(x_train), np.array(y_train), epochs=50, batch_size=32)  
score = model.evaluate(np.array(x_train), np.array(y_train), batch_size=32)
print("Train accuracy:", score[1])
score = model.evaluate(np.array(x_test), np.array(y_test), batch_size=32)
print("Test accuracy:", score[1])

Epoch 1/50
1004/1004 [==============================] - 3s 3ms/step - loss: 5.5439 - acc: 0.0030
Epoch 2/50
1004/1004 [==============================] - 1s 618us/step - loss: 5.3302 - acc: 0.0219
Epoch 3/50
1004/1004 [==============================] - 1s 596us/step - loss: 4.1412 - acc: 0.0857
Epoch 4/50
1004/1004 [==============================] - 1s 637us/step - loss: 3.1156 - acc: 0.2191
Epoch 5/50
1004/1004 [==============================] - 1s 623us/step - loss: 2.5531 - acc: 0.2968 0s - loss: 2.6004 - acc
Epoch 6/50
1004/1004 [==============================] - 1s 663us/step - loss: 2.2142 - acc: 0.3586
Epoch 7/50
1004/1004 [==============================] - 1s 748us/step - loss: 1.9747 - acc: 0.4183
Epoch 8/50
1004/1004 [==============================] - 1s 675us/step - loss: 1.8442 - acc: 0.4512
Epoch 9/50
1004/1004 [==============================] - 1s 634us/step - loss: 1.6607 - acc: 0.4861
Epoch 10/50
1004/1004 [==============================] - 1s 802us/step - loss: 1.6243 -

Train your best network on inputs from the data from at least 2 different fonts. How does your accuracy compare to the 1-font case?

In [184]:
train_paths = ["fonts/AGENCY.csv", "fonts/BELL.csv", "fonts/TXT.csv"]
test_paths = ["fonts/AGENCY.csv", "fonts/BELL.csv", "fonts/TXT.csv"]
X, y = import_data(train_paths)
unique_chars = list(set(y))
y = convert_to_one_hot(y, unique_chars)

model = Sequential()
# Layer 1: Convolution 2D layer with relu activations
model.add(Conv2D(32, (4, 4), activation='relu', input_shape=(20, 20, 1)))
# Layer 2: Max pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# Layer 3: Convolution
model.add(Conv2D(64, (4, 4), activation='relu'))
# Layer 4: Max pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# Layer 5: Flatten
model.add(Flatten())
# Layer 6: Dropout
model.add(Dropout(0.4))
# Layer 7: Dense with relu activation
model.add(Dense(1024, activation='relu'))
# Layer 8: Dense with softmax activation
model.add(Dense(len(unique_chars), activation='softmax'))

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.3)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(np.array(x_train), np.array(y_train), epochs=50, batch_size=32)  
score = model.evaluate(np.array(x_train), np.array(y_train), batch_size=32)
print("Train accuracy:", score[1])
score = model.evaluate(np.array(x_test), np.array(y_test), batch_size=32)
print("Test accuracy:", score[1])

Epoch 1/50
2287/2287 [==============================] - 4s 2ms/step - loss: 5.8043 - acc: 0.0105
Epoch 2/50
2287/2287 [==============================] - 2s 670us/step - loss: 4.8513 - acc: 0.0621
Epoch 3/50
2287/2287 [==============================] - 2s 662us/step - loss: 3.6822 - acc: 0.1740
Epoch 4/50
2287/2287 [==============================] - 2s 671us/step - loss: 3.0136 - acc: 0.2571
Epoch 5/50
2287/2287 [==============================] - 2s 665us/step - loss: 2.5779 - acc: 0.3209
Epoch 6/50
2287/2287 [==============================] - 2s 681us/step - loss: 2.2890 - acc: 0.3704
Epoch 7/50
2287/2287 [==============================] - 2s 660us/step - loss: 2.0679 - acc: 0.4066
Epoch 8/50
2287/2287 [==============================] - 2s 670us/step - loss: 1.8940 - acc: 0.4464
Epoch 9/50
2287/2287 [==============================] - 2s 701us/step - loss: 1.7406 - acc: 0.4958
Epoch 10/50
2287/2287 [==============================] - 2s 724us/step - loss: 1.6381 - acc: 0.5059 1s - l
Epoc

What accuracy do you see when testing with inputs from a font you didn't train on now?

In [185]:
train_paths = ["fonts/AGENCY.csv", "fonts/BELL.csv", "fonts/TXT.csv"]
test_paths = ["fonts/SNAP.csv"]
x_train, y_train = import_data(train_paths)
x_test, y_test = import_data(test_paths)
unique_chars = list(set(y_train).union(y_test))
y_train = convert_to_one_hot(y_train, unique_chars)
y_test = convert_to_one_hot(y_test, unique_chars)

model = Sequential()
# Layer 1: Convolution 2D layer with relu activations
model.add(Conv2D(32, (4, 4), activation='relu', input_shape=(20, 20, 1)))
# Layer 2: Max pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# Layer 3: Convolution
model.add(Conv2D(64, (4, 4), activation='relu'))
# Layer 4: Max pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# Layer 5: Flatten
model.add(Flatten())
# Layer 6: Dropout
model.add(Dropout(0.4))
# Layer 7: Dense with relu activation
model.add(Dense(1024, activation='relu'))
# Layer 8: Dense with softmax activation
model.add(Dense(len(unique_chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(np.array(x_train), np.array(y_train), epochs=50, batch_size=32)  
score = model.evaluate(np.array(x_train), np.array(y_train), batch_size=32)
print("Train accuracy:", score[1])
score = model.evaluate(np.array(x_test), np.array(y_test), batch_size=32)
print("Test accuracy:", score[1])

Epoch 1/50
3268/3268 [==============================] - 4s 1ms/step - loss: 5.6857 - acc: 0.0125A: 4s - loss
Epoch 2/50
3268/3268 [==============================] - 3s 830us/step - loss: 4.1876 - acc: 0.1166 1s 
Epoch 3/50
3268/3268 [==============================] - 2s 757us/step - loss: 3.1806 - acc: 0.2252 0s - loss: 3.2080 - acc: 
Epoch 4/50
3268/3268 [==============================] - 3s 862us/step - loss: 2.6029 - acc: 0.3008
Epoch 5/50
3268/3268 [==============================] - 2s 676us/step - loss: 2.2586 - acc: 0.3721
Epoch 6/50
3268/3268 [==============================] - 3s 919us/step - loss: 2.0292 - acc: 0.4189
Epoch 7/50
3268/3268 [==============================] - 3s 835us/step - loss: 1.8490 - acc: 0.4446 0s - loss: 1.8307 - acc
Epoch 8/50
3268/3268 [==============================] - 3s 878us/step - loss: 1.7105 - acc: 0.4783
Epoch 9/50
3268/3268 [==============================] - 2s 736us/step - loss: 1.5781 - acc: 0.5098
Epoch 10/50
3268/3268 [======================